# 📈 Logistic Growth Model of Phytoplankton Based on Lighting Duration

This notebook simulates phytoplankton population growth using a logistic growth model, based on varying light exposure durations between 6 and 24 hours.

🧪 **How it works:**
The model estimates population growth based on three logistic parameters (`K`, `r`, and `t₀`) which are dynamically interpolated according to the chosen light duration. These predictions are then visually compared against actual population data collected for 6h, 12h, and 24h light exposures.

---

### 📊 Features:
- 🔄 **Interactive simulation** using a slider (6–24 hours)
- 📈 **Logistic curve prediction** for user-selected light duration
- 🧾 **Comparison with actual data** for 6h, 12h, and 24h durations
- 📉 **MAPE error analysis** (Mean Absolute Percentage Error)
- 💡 **Interpretation** of which actual exposure the prediction is closest to
- 📊 **Bar chart of errors** for quick visual insight
- 🔎 No external Excel file needed — data is fully embedded in the notebook
- ✅ Built with NumPy, Matplotlib, Pandas, scikit-learn, and ipywidgets

---

### 👨‍💻 Developed by:
**Yoga Andriyanto**  
📅 July 2025


In [4]:
# Install dependencies (only needed in Colab)
!pip install ipywidgets openpyxl --quiet

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact
from sklearn.metrics import mean_absolute_percentage_error

# Logistic model functions
def calculate_K(h, K_6, K_12, K_24):
    if h == 6:
        return K_6
    elif h == 12:
        return K_12
    elif h == 24:
        return K_24
    elif 6 < h < 12:
        return K_6 + (K_12 - K_6) / 6 * (h - 6)
    elif 12 < h < 24:
        return K_12 + (K_24 - K_12) / 12 * (h - 12)
    else:
        raise ValueError("Invalid light duration h. Must be between 6–24.")

def calculate_r(h, a, b):
    return a * (1 - np.exp(-b * h))

def calculate_t0(h, p, q):
    return p * (1 - np.exp(-q * h))

def logistic_growth(t, K, r, t0):
    return K / (1 + np.exp(-r * (t - t0)))


In [5]:
# Manual input of population data (Day 1–9 for each light duration)
data_all = np.array([
    # 6 hours
    [1, 700000, 6], [2, 1050000, 6], [3, 1166666.667, 6], [4, 1255555.556, 6],
    [5, 1483333.333, 6], [6, 1544444.444, 6], [7, 1577777.778, 6], [8, 2300000, 6], [9, 2244444.444, 6],
    # 12 hours
    [1, 700000, 12], [2, 1083333.333, 12], [3, 1727777.778, 12], [4, 1916666.667, 12],
    [5, 2494444.444, 12], [6, 2833333.333, 12], [7, 3133333.333, 12], [8, 4272222.222, 12], [9, 3661111.111, 12],
    # 24 hours
    [1, 700000, 24], [2, 1161111.111, 24], [3, 2366666.667, 24], [4, 3394444.444, 24],
    [5, 4027777.778, 24], [6, 5119444.444, 24], [7, 5350000, 24], [8, 7219444.444, 24], [9, 5444444.444, 24]
])

# Convert to DataFrame
df = pd.DataFrame(data_all, columns=["Day", "Density", "Duration"])
df_pivot = df.pivot(index="Day", columns="Duration", values="Density").rename(
    columns={6: "6 Hour", 12: "12 Hour", 24: "24 Hour"}
)
days = df_pivot.index


In [6]:
# Model and visualization function
def interactive_model(h_input):
    # Parameters
    K_6 = 2530000.000
    K_12 = 4699444.444
    K_24 = 7941388.889
    a = 0.61999296
    b = 0.12328981
    p = 3.87674621
    q = 0.18575130

    # Calculate model output
    K_new = calculate_K(h_input, K_6, K_12, K_24)
    r_new = calculate_r(h_input, a, b)
    t0_new = calculate_t0(h_input, p, q)
    predictions = logistic_growth(days - 1, K_new, r_new, t0_new)

    # Calculate errors
    mape_6 = mean_absolute_percentage_error(df_pivot["6 Hour"], predictions) * 100
    mape_12 = mean_absolute_percentage_error(df_pivot["12 Hour"], predictions) * 100
    mape_24 = mean_absolute_percentage_error(df_pivot["24 Hour"], predictions) * 100
    mape_values = [mape_6, mape_12, mape_24]
    mape_labels = ['6h', '12h', '24h']

    # Plotting
    fig, axs = plt.subplots(2, 1, figsize=(8, 8))

    # Logistic Curve
    axs[0].scatter(days, df_pivot["6 Hour"], color='red', label='Actual Data 6 Hours')
    axs[0].scatter(days, df_pivot["12 Hour"], color='blue', label='Actual Data 12 Hours')
    axs[0].scatter(days, df_pivot["24 Hour"], color='green', label='Actual Data 24 Hours')
    axs[0].plot(days, predictions, color='black', linewidth=2, label=f'Logistic Prediction {h_input} Hours')
    axs[0].set_title(f'Logistic Growth Curve - {h_input} Hours Light Duration')
    axs[0].set_xlabel('Day')
    axs[0].set_ylabel('Density (cells/mL)')
    axs[0].legend()
    axs[0].grid(True)

    # Error bar
    axs[1].bar(mape_labels, mape_values, color=['red', 'blue', 'green'])
    axs[1].set_title('MAPE Compared to Actual Data')
    axs[1].set_ylabel('MAPE (%)')
    axs[1].grid(axis='y')

    plt.tight_layout()
    plt.show()

    # Print model parameters
    print(f"K value for {h_input} hours:  {K_new:.2f}")
    print(f"r value for {h_input} hours:  {r_new:.5f}")
    print(f"t0 value for {h_input} hours: {t0_new:.5f}")
    print("\nMAPE Compared to Actual Data:")
    print(f"- vs 6h  → {mape_6:.2f}%")
    print(f"- vs 12h → {mape_12:.2f}%")
    print(f"- vs 24h → {mape_24:.2f}%")

    # Interpretation
    errors = {'6h': mape_6, '12h': mape_12, '24h': mape_24}
    sorted_errors = sorted(errors.items(), key=lambda x: x[1])
    best_match, best_error = sorted_errors[0]
    second_best_match, second_best_error = sorted_errors[1]

    print("\n📌 Interpretation:")
    if abs(best_error - second_best_error) < 2:
        print(f"The prediction for {h_input} hours lies between {best_match} and {second_best_match} exposure.")
    else:
        print(f"The prediction for {h_input} hours is most similar to actual {best_match} exposure data.")


In [7]:
# Create interactive slider under the chart
interact(interactive_model, h_input=(6, 24, 1))


interactive(children=(IntSlider(value=15, description='h_input', max=24, min=6), Output()), _dom_classes=('wid…

<function __main__.interactive_model(h_input)>